In [1]:
import pandas as pd
import numpy as np
import nltk
import pickle
from collections import Counter
from textblob import TextBlob
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
from wordcloud import WordCloud, ImageColorGenerator
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import pandas as pd
import numpy as np
import ast

In [3]:
df = pd.read_csv('alldata 0802.csv')

df['public_metrics'][0]

"{'retweet_count': 76, 'reply_count': 5, 'like_count': 311, 'quote_count': 6}"

In [4]:
def res(string_list):
    return ast.literal_eval(string_list)

df['public_metrics'] = df['public_metrics'].apply(res)

In [5]:
df.columns

Index(['Unnamed: 0', 'created_at', 'text', 'entities', 'public_metrics',
       'union name', 'Relevance:blank = relevant, 1=irrelevant, 2=discuss'],
      dtype='object')

In [6]:
df = df[['created_at','text', 'entities','public_metrics','union name']]


In [7]:
df

,created_at,text,entities,public_metrics,union name
0,2022-06-08T01:39:00.000Z,"Every student in America—Black, brown or white...","{'annotations': [{'start': 17, 'end': 23, 'pro...","{'retweet_count': 76, 'reply_count': 5, 'like_...",NEAToday
1,2022-05-20T00:10:00.000Z,"So far, education advocates have not had a cle...","{'urls': [{'start': 109, 'end': 132, 'url': 'h...","{'retweet_count': 3, 'reply_count': 1, 'like_c...",NEAToday
2,2022-05-19T17:07:04.000Z,When educators and management come together us...,"{'urls': [{'start': 164, 'end': 187, 'url': 'h...","{'retweet_count': 6, 'reply_count': 1, 'like_c...",NEAToday
3,2022-05-09T17:47:01.000Z,The mental health of our students must be a to...,"{'annotations': [{'start': 202, 'end': 227, 'p...","{'retweet_count': 15, 'reply_count': 4, 'like_...",NEAToday
4,2022-04-22T19:46:02.000Z,One way to honor #EarthDay: Introduce your stu...,"{'hashtags': [{'start': 17, 'end': 26, 'tag': ...","{'retweet_count': 15, 'reply_count': 2, 'like_...",NEAToday
...,...,...,...,...,...
11960,2012-03-12T19:45:04.000Z,Unite Here Local 427 in Missoula wants a promi...,"{'hashtags': [{'start': 93, 'end': 96, 'tag': ...","{'retweet_count': 1, 'reply_count': 0, 'like_c...",unithere
11961,2012-03-06T21:50:04.000Z,100 years before OSHA - these kids were workin...,"{'urls': [{'start': 85, 'end': 105, 'url': 'ht...","{'retweet_count': 1, 'reply_count': 0, 'like_c...",unithere
11962,2012-02-27T21:35:06.000Z,WOO! New Contract Ratified at Consol Energy C...,"{'hashtags': [{'start': 53, 'end': 56, 'tag': ...","{'retweet_count': 0, 'reply_count': 0, 'like_c...",unithere
11963,2012-02-11T00:29:24.000Z,Regional director Shawn Bagley isn't just a co...,"{'urls': [{'start': 115, 'end': 135, 'url': 'h...","{'retweet_count': 1, 'reply_count': 0, 'like_c...",unithere


In [8]:
#often, we want to remove stopwords
stopwords = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 
                     'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 
                     'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 
                     'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 
                     'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 
                     'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 
                     'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 
                     'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 
                     'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 
                     'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 
                     'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 'can', 'will',
                     'just', 'dont', 'should', 'aint', 'arent', 'couldn', 'could', 'would', 'much', 'must',
                     'didnt', 'doesnt', 'hadnt', 'hasnt', 'havent', 'isnt', 'mightnt', 'mustnt', 'neednt', 'shan',
                     'shouldnt', 'wasnt', 'werent', 'wont', 'wouldnt','rt','mt','amp','mil','ve','don','s','st','pres','req','aft','ua']

In [9]:
 #adopted from catriscode, https://catriscode.com/2021/05/01/tweets-cleaning-with-python/

import numpy as np
import re

def clean_tweet(tweet):
    raw_tweet = str(tweet)
    if type(tweet) == float:
        return ""
    temp = re.sub("'", "", str(tweet)) # to avoid removing contractions in english
    temp = temp.lower()
    temp = re.sub("@[A-Za-z0-9_]+","", temp)
    temp = re.sub("#[A-Za-z0-9_]+","", temp)
    temp = re.sub(r'http\S+', '', temp)
    temp = re.sub(r"www.\S+", "", temp)
    temp = re.sub('[()!?]', ' ', temp)
    temp = re.sub('\[.*?\]',' ', temp)
    #temp = re.sub("[^a-z0-9]"," ", temp) # this removes all letters including CO2
    temp = re.sub("[^a-z]"," ", temp)
    temp = temp.split()
    temp = [w for w in temp if not w in stopwords]
    temp = " ".join(word for word in temp)
    
    if temp is None:
        print (raw_tweet,temp)
        
    return temp

In [10]:
df['clean_text'] = df['text'].apply(clean_tweet)

In [11]:
df = df.dropna(subset=['clean_text'])
df
#empty data with cleaning procedure

#`reset_index()`will recreate index column every time we run it on same data 
#`drop = True` paramater won’t create that as column in the dataframe, look at the difference between following two dataset
df = df.reset_index(drop=False) 

In [12]:
df

,index,created_at,text,entities,public_metrics,union name,clean_text
0,0,2022-06-08T01:39:00.000Z,"Every student in America—Black, brown or white...","{'annotations': [{'start': 17, 'end': 23, 'pro...","{'retweet_count': 76, 'reply_count': 5, 'like_...",NEAToday,every student america black brown white native...
1,1,2022-05-20T00:10:00.000Z,"So far, education advocates have not had a cle...","{'urls': [{'start': 109, 'end': 132, 'url': 'h...","{'retweet_count': 3, 'reply_count': 1, 'like_c...",NEAToday,far education advocates clear role addressing ...
2,2,2022-05-19T17:07:04.000Z,When educators and management come together us...,"{'urls': [{'start': 164, 'end': 187, 'url': 'h...","{'retweet_count': 6, 'reply_count': 1, 'like_c...",NEAToday,educators management come together using susta...
3,3,2022-05-09T17:47:01.000Z,The mental health of our students must be a to...,"{'annotations': [{'start': 202, 'end': 227, 'p...","{'retweet_count': 15, 'reply_count': 4, 'like_...",NEAToday,mental health students top priority school com...
4,4,2022-04-22T19:46:02.000Z,One way to honor #EarthDay: Introduce your stu...,"{'hashtags': [{'start': 17, 'end': 26, 'tag': ...","{'retweet_count': 15, 'reply_count': 2, 'like_...",NEAToday,one way honor introduce students girls green w...
...,...,...,...,...,...,...,...
11960,11960,2012-03-12T19:45:04.000Z,Unite Here Local 427 in Missoula wants a promi...,"{'hashtags': [{'start': 93, 'end': 96, 'tag': ...","{'retweet_count': 1, 'reply_count': 0, 'like_c...",unithere,unite local missoula wants promise good jobs e...
11961,11961,2012-03-06T21:50:04.000Z,100 years before OSHA - these kids were workin...,"{'urls': [{'start': 85, 'end': 105, 'url': 'ht...","{'retweet_count': 1, 'reply_count': 0, 'like_c...",unithere,years osha kids working mines
11962,11962,2012-02-27T21:35:06.000Z,WOO! New Contract Ratified at Consol Energy C...,"{'hashtags': [{'start': 53, 'end': 56, 'tag': ...","{'retweet_count': 0, 'reply_count': 0, 'like_c...",unithere,woo new contract ratified consol energy center
11963,11963,2012-02-11T00:29:24.000Z,Regional director Shawn Bagley isn't just a co...,"{'urls': [{'start': 115, 'end': 135, 'url': 'h...","{'retweet_count': 1, 'reply_count': 0, 'like_c...",unithere,regional director shawn bagley cop hes cop jus...


In [13]:
df['clean_text_list'] = [l.split() for l in df['clean_text']]

In [14]:
df

,index,created_at,text,entities,public_metrics,union name,clean_text,clean_text_list
0,0,2022-06-08T01:39:00.000Z,"Every student in America—Black, brown or white...","{'annotations': [{'start': 17, 'end': 23, 'pro...","{'retweet_count': 76, 'reply_count': 5, 'like_...",NEAToday,every student america black brown white native...,"[every, student, america, black, brown, white,..."
1,1,2022-05-20T00:10:00.000Z,"So far, education advocates have not had a cle...","{'urls': [{'start': 109, 'end': 132, 'url': 'h...","{'retweet_count': 3, 'reply_count': 1, 'like_c...",NEAToday,far education advocates clear role addressing ...,"[far, education, advocates, clear, role, addre..."
2,2,2022-05-19T17:07:04.000Z,When educators and management come together us...,"{'urls': [{'start': 164, 'end': 187, 'url': 'h...","{'retweet_count': 6, 'reply_count': 1, 'like_c...",NEAToday,educators management come together using susta...,"[educators, management, come, together, using,..."
3,3,2022-05-09T17:47:01.000Z,The mental health of our students must be a to...,"{'annotations': [{'start': 202, 'end': 227, 'p...","{'retweet_count': 15, 'reply_count': 4, 'like_...",NEAToday,mental health students top priority school com...,"[mental, health, students, top, priority, scho..."
4,4,2022-04-22T19:46:02.000Z,One way to honor #EarthDay: Introduce your stu...,"{'hashtags': [{'start': 17, 'end': 26, 'tag': ...","{'retweet_count': 15, 'reply_count': 2, 'like_...",NEAToday,one way honor introduce students girls green w...,"[one, way, honor, introduce, students, girls, ..."
...,...,...,...,...,...,...,...,...
11960,11960,2012-03-12T19:45:04.000Z,Unite Here Local 427 in Missoula wants a promi...,"{'hashtags': [{'start': 93, 'end': 96, 'tag': ...","{'retweet_count': 1, 'reply_count': 0, 'like_c...",unithere,unite local missoula wants promise good jobs e...,"[unite, local, missoula, wants, promise, good,..."
11961,11961,2012-03-06T21:50:04.000Z,100 years before OSHA - these kids were workin...,"{'urls': [{'start': 85, 'end': 105, 'url': 'ht...","{'retweet_count': 1, 'reply_count': 0, 'like_c...",unithere,years osha kids working mines,"[years, osha, kids, working, mines]"
11962,11962,2012-02-27T21:35:06.000Z,WOO! New Contract Ratified at Consol Energy C...,"{'hashtags': [{'start': 53, 'end': 56, 'tag': ...","{'retweet_count': 0, 'reply_count': 0, 'like_c...",unithere,woo new contract ratified consol energy center,"[woo, new, contract, ratified, consol, energy,..."
11963,11963,2012-02-11T00:29:24.000Z,Regional director Shawn Bagley isn't just a co...,"{'urls': [{'start': 115, 'end': 135, 'url': 'h...","{'retweet_count': 1, 'reply_count': 0, 'like_c...",unithere,regional director shawn bagley cop hes cop jus...,"[regional, director, shawn, bagley, cop, hes, ..."


In [15]:
# save dataframe
df.to_csv('All Tweets Twitter Cleaned Text.csv')

In [16]:
words = df['clean_text_list']

words

0        [every, student, america, black, brown, white,...
1        [far, education, advocates, clear, role, addre...
2        [educators, management, come, together, using,...
3        [mental, health, students, top, priority, scho...
4        [one, way, honor, introduce, students, girls, ...
                               ...                        
11960    [unite, local, missoula, wants, promise, good,...
11961                  [years, osha, kids, working, mines]
11962    [woo, new, contract, ratified, consol, energy,...
11963    [regional, director, shawn, bagley, cop, hes, ...
11964    [points, health, safety, critical, issues, man...
Name: clean_text_list, Length: 11965, dtype: object

In [19]:
import gensim
from gensim import corpora
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

dictionary = corpora.Dictionary(words)
corpus = [dictionary.doc2bow(word) for word in words]

# define the number of topics to identify
num_topics = 7


# build the LDA model
lda_model = gensim.models.LdaModel(corpus=corpus,
                                   id2word=dictionary,
                                   num_topics=num_topics,
                                   random_state=100,
                                   update_every=1,
                                   chunksize=10,
                                   passes=10,
                                   alpha='auto',
                                   per_word_topics=True)

# visualize the topics using pyLDAvis
vis_data = gensimvis.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(vis_data)

# print the top words for each topic
for topic in lda_model.print_topics():
    print(topic)

/Users/jeffreypan/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/Users/jeffreypan/opt/anaconda3/lib/python3.8/site-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


(0, '0.034*"choice" + 0.025*"say" + 0.025*"time" + 0.025*"environment" + 0.024*"now" + 0.021*"strike" + 0.019*"action" + 0.017*"support" + 0.015*"video" + 0.014*"unite"')
(1, '0.044*"people" + 0.043*"coal" + 0.039*"work" + 0.024*"get" + 0.024*"need" + 0.021*"miners" + 0.020*"make" + 0.015*"day" + 0.014*"environmental" + 0.013*"choice"')
(2, '0.111*"via" + 0.031*"sustainability" + 0.029*"t" + 0.017*"many" + 0.017*"create" + 0.016*"housekeepers" + 0.016*"week" + 0.015*"report" + 0.014*"read" + 0.013*"system"')
(3, '0.052*"union" + 0.048*"safety" + 0.045*"jobs" + 0.037*"good" + 0.035*"working" + 0.033*"health" + 0.032*"climate" + 0.022*"check" + 0.020*"real" + 0.017*"today"')
(4, '0.056*"green" + 0.040*"heat" + 0.039*"food" + 0.032*"sustainable" + 0.022*"labor" + 0.021*"job" + 0.019*"new" + 0.017*"future" + 0.013*"families" + 0.013*"unions"')
(5, '0.144*"workers" + 0.021*"water" + 0.020*"energy" + 0.016*"us" + 0.014*"local" + 0.012*"w" + 0.011*"together" + 0.011*"join" + 0.011*"years" + 0

In [20]:
pyLDAvis.display(vis_data)

/Users/jeffreypan/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [23]:
df.columns

/Users/jeffreypan/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Index(['index', 'created_at', 'text', 'entities', 'public_metrics',
       'union name', 'clean_text', 'clean_text_list'],
      dtype='object')

In [24]:
df.head()

/Users/jeffreypan/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,index,created_at,text,entities,public_metrics,union name,clean_text,clean_text_list
0,0,2022-06-08T01:39:00.000Z,"Every student in America—Black, brown or white...","{'annotations': [{'start': 17, 'end': 23, 'pro...","{'retweet_count': 76, 'reply_count': 5, 'like_...",NEAToday,every student america black brown white native...,"[every, student, america, black, brown, white,..."
1,1,2022-05-20T00:10:00.000Z,"So far, education advocates have not had a cle...","{'urls': [{'start': 109, 'end': 132, 'url': 'h...","{'retweet_count': 3, 'reply_count': 1, 'like_c...",NEAToday,far education advocates clear role addressing ...,"[far, education, advocates, clear, role, addre..."
2,2,2022-05-19T17:07:04.000Z,When educators and management come together us...,"{'urls': [{'start': 164, 'end': 187, 'url': 'h...","{'retweet_count': 6, 'reply_count': 1, 'like_c...",NEAToday,educators management come together using susta...,"[educators, management, come, together, using,..."
3,3,2022-05-09T17:47:01.000Z,The mental health of our students must be a to...,"{'annotations': [{'start': 202, 'end': 227, 'p...","{'retweet_count': 15, 'reply_count': 4, 'like_...",NEAToday,mental health students top priority school com...,"[mental, health, students, top, priority, scho..."
4,4,2022-04-22T19:46:02.000Z,One way to honor #EarthDay: Introduce your stu...,"{'hashtags': [{'start': 17, 'end': 26, 'tag': ...","{'retweet_count': 15, 'reply_count': 2, 'like_...",NEAToday,one way honor introduce students girls green w...,"[one, way, honor, introduce, students, girls, ..."
